In [31]:
import os
os.chdir('/content/drive/MyDrive/000GithubRepos/bigcontest2024')

In [32]:
pip install python-dotenv

In [33]:
pip install langchain_community

In [34]:
pip install sentence-transformers

In [35]:
pip install neo4j

In [36]:
pip install neo4j_genai

In [37]:
pip install openai

In [38]:
pip install langchain_huggingface

In [39]:
pip install langchain_google_genai

In [40]:
pip install openai

In [41]:
from llm_response.get_llm_model import get_llm_model

llm = get_llm_model()

In [42]:
from cypher_query.retrieval_query import retrievalQuery, retrievalQuery_v2
from dotenv import load_dotenv
from config import CONFIG

from graphrag.get_embedding_model import get_embedding_model
from langchain_community.vectorstores.neo4j_vector import Neo4jVector

load_dotenv()

True

In [43]:
os.getenv('NEO4J_URI')

'bolt://43.201.102.234/:7687'

In [44]:
retrievalQuery_v3 = """
MATCH (node)<-[:HAS_REVIEW]-(store)
RETURN node.text AS text,
       store AS store,
       score,
       {
         pk : store.pk,
         reviewText: node.text,
         storeName: store.MCT_NM,
         store_Type: store.MCT_TYPE,
         store_Image: {kakao: store.image_url_kakao, google: store.image_url_google},
         store_Rating: {kakao: store.rating_kakao, google: store.rating_google},
         reviewCount: {kakao: store.rating_count_kakao, google: store.rating_count_google},
         purpose: store.purpose,
         use_how: store.use_how,
         viwit_with: store.visit_with,
         wait_time: store.wait_time,
         menu : store.menu
       } AS metadata
"""

In [45]:
def get_neo4j_vector(index_name='queryVector'):
    neo4jvector = Neo4jVector.from_existing_index(
        embedding=get_embedding_model(),  # Using the custom embedding function
        url=CONFIG.neo4j_url,
        database='neo4j',
        username=CONFIG.neo4j_user,
        password=CONFIG.neo4j_password,
        index_name=index_name,
        text_node_property="textEmbedding",
        retrieval_query=retrievalQuery_v3
    )
    return neo4jvector

def retrieve_store_nodes(query):
    """
    쿼리 임베딩 <-> 리뷰 임베딩 유사도 측정하여 상위 n개의 Store 노드 추출
    """
    store_retriever = get_neo4j_vector().as_retriever(search_kwargs={"k": 6})
    result_nodes = store_retriever.invoke(query)  # invoke는 동기적으로 실행되는 메서드
    return result_nodes

store_retriever = get_neo4j_vector().as_retriever(search_kwargs={"k": 5})
vs = get_neo4j_vector()

In [46]:
query = "8살 아이 포함 3인 가족 가기 좋은 평균가격 3만원대 패밀리 레스토랑 추천"
query = "20대 후반 여자 또래 친구끼리 가기 좋은 카페 추천해줘"

In [47]:
store_retriever_rev_emb = get_neo4j_vector().as_retriever(search_kwargs={"k": 10})

In [48]:
rev_sim_result = store_retriever_rev_emb.invoke(query)

In [49]:

def get_candidate_str(candidates):
    drop_dup = []
    for r in candidates:
        if r.metadata['pk'] not in [d.metadata['pk'] for d in drop_dup]:
            drop_dup.append(r)
    drop_dup

    candidates_str = ''
    for d in drop_dup:
        # 가게명
        candidates_str += f"가게명 : {d.metadata['storeName']}\n"
        # pk
        candidates_str += f"pk : {d.metadata['pk']}\n"
        # 리뷰
        candidates_str += f"리뷰 : {d.metadata['reviewText']}\n"
        # 평점
        ratings_lst = []
        for platform in ['naver', 'kakao', 'google']:
            if (platform in d.metadata['store_Rating']) and (d.metadata['store_Rating'][platform] is not None):
                pf_rating = d.metadata['store_Rating'][platform]
            else:
                continue
            if platform in d.metadata['reviewCount'] and (d.metadata['reviewCount'][platform] is not None):
                pf_rc = d.metadata['reviewCount'][platform]
            else:
                continue
            ratings_lst.append(f"{platform} {pf_rating}({pf_rc}명)")
        rating_str = ', '.join(ratings_lst)
        candidates_str += f"평점 : {rating_str}\n"
        # 음식 유형
        if 'store_Type' in d.metadata:
            candidates_str += f"음식 유형 : {d.metadata['store_Type']}\n"
        # 방문 목적 top 3
        if 'purpose' in d.metadata:
            candidates_str += f"방문 목적 top 3 : {d.metadata['purpose']}\n"
        # 대기 시간 통계
        if 'wait_time' in d.metadata:
            wait_time_str = d.metadata['wait_time'].replace('{', '').replace('}', '').replace('"', '')
            candidates_str += f"대기 시간 통계 : {wait_time_str}\n"
        # 예약 필요 여부
        if 'use_how' in d.metadata:
            use_how_str = d.metadata['use_how'].replace('{', '').replace('}', '').replace('"', '')
            candidates_str += f"예약 필요 여부 통계 : {use_how_str}\n"
        # 메뉴
        if 'menu' in d.metadata:
            candidates_str += f"메뉴 : {d.metadata['menu']}\n"

        candidates_str += '\n'
    return candidates_str


In [50]:
candidates_str = get_candidate_str(rev_sim_result)

In [51]:
print(candidates_str)

가게명 : 우호적무관심
pk : 5938
리뷰 : 만약 여자친구가 있었더라면
참 좋아했을것 같은 카페다
평점 : kakao 4.0(49명)
음식 유형 : 단품요리 전문
방문 목적 top 3 : 여행, 데이트, 나들이
대기 시간 통계 : 10분 이내:1,바로 입장:43
예약 필요 여부 통계 : 예약 없이 이용:42,포장·배달 이용:1
메뉴 : 구좌 당근주스:7000

가게명 : 라토커피제주
pk : 2156
리뷰 : 분위기 좋은카페. 연인이랑같이
평점 : google 3.9(191명)
음식 유형 : 단품요리 전문
방문 목적 top 3 : 여행, 데이트, 일상
대기 시간 통계 : 바로 입장:14
예약 필요 여부 통계 : 예약 없이 이용:14
메뉴 : 넛츠끄레마:6500, 크림브륄레:7000, 딥초코:6500, 약과쿠키:4500, 약과버터바:6000, 약과휘낭시에:4000, 바스크치즈케이크:6500, 크로플:10000

가게명 : 블랙이쉬레드
pk : 3513
리뷰 : 공간이 굉장히 넓었습니다.
갤러리 같은 분위기의 카페입니다.
단체로 가기에도 좋아요.
평점 : google 4.6(76명)
음식 유형 : 단품요리 전문
방문 목적 top 3 : 여행, 데이트, 일상
대기 시간 통계 : 바로 입장:69
예약 필요 여부 통계 : 예약 없이 이용:67
메뉴 : 우바밀크티:7500, 홍차:6000, 홍차 아이스티 (레몬, 자몽):6500, 소금빵:4000, 대파 치즈 크로와상:5000, 더블베리 아몬드 크로와상:5000, 초코 크로와상:5000, 쑥 밀크티:8000, 아메리카노:6000, 디카페인 아메리카노:6500, 카페라떼:6500, 유수암라떼 (10온스):7000, 콜드브루:6300, 콜드브루 라떼:6800, 푸딩:6000

가게명 : 동백정원
pk : 1927
리뷰 : 분위기 좋은 카페
평점 : kakao 4.8(54명), google 4.5(152명)
음식 유형 : 단품요리 전문
방문 목적 top 3 : 여행, 데이트, 나들이
대기 시간 통계 : 바로 입장:24
예

In [52]:
from prompt.final_selecting_for_recomm import FINAL_SELECTING_FOR_RECOMM, FINAL_SELECTING_FOR_RECOMM_v2

In [53]:
response = llm.invoke(FINAL_SELECTING_FOR_RECOMM_v2.format(candidates=candidates_str, query=query, intent=''))

In [54]:
response.usage_metadata['input_tokens']

4498

In [55]:
eval(response.content.replace('```', '').replace('json', '').strip())

{'decorational_mention_start': '20대 후반 여자 친구들끼리 가기 좋은 카페를 찾으시는군요! 분위기 좋고, 편안하게 이야기 나누며 시간 보내기 좋은 곳들을 추천해 드릴게요.',
 'recommendations': [{'pk': 3513,
   'desc': "넓고 갤러리 같은 분위기의 '블랙이쉬레드'는 친구들과 편안하게 이야기 나누기에 좋은 공간이에요. 다양한 종류의 홍차와 커피, 그리고 맛있는 베이커리도 준비되어 있어, 취향에 맞게 즐길 수 있답니다. 특히 단체로 가기에도 좋고, 대기 시간도 거의 없어 편리하게 이용할 수 있어요."},
  {'pk': 5938,
   'desc': "'우호적무관심'은 데이트 장소로도 인기 있는 곳이지만, 친구들과 조용하고 편안하게 이야기 나누기에도 좋아요. 특히 구좌 당근 주스는 이곳의 시그니처 메뉴로, 맛도 좋고 분위기와 잘 어울린답니다. 대기 시간도 짧고, 예약 없이 바로 이용할 수 있어 편리해요."},
  {'pk': 2156,
   'desc': "'라토커피제주'는 분위기 좋은 카페로, 친구들과 함께 맛있는 디저트를 즐기며 이야기꽃을 피우기에 좋습니다. 특히 약과 쿠키, 약과 버터바, 약과 휘낭시에 등 독특한 디저트 메뉴들이 인기가 많아요. 대기 시간도 짧고, 예약 없이 바로 이용할 수 있어 편리하게 방문할 수 있습니다."}],
 'decorational_mention_end': '세 곳 모두 20대 후반 여성분들이 좋아할 만한 분위기와 메뉴를 갖추고 있어요. 친구들과 즐거운 시간 보내세요!'}

In [56]:
print(response.content)

```json
{
    "decorational_mention_start": "20대 후반 여자 친구들끼리 가기 좋은 카페를 찾으시는군요! 분위기 좋고, 편안하게 이야기 나누며 시간 보내기 좋은 곳들을 추천해 드릴게요.",
    "recommendations": [
        {
            "pk": 3513,
            "desc": "넓고 갤러리 같은 분위기의 '블랙이쉬레드'는 친구들과 편안하게 이야기 나누기에 좋은 공간이에요. 다양한 종류의 홍차와 커피, 그리고 맛있는 베이커리도 준비되어 있어, 취향에 맞게 즐길 수 있답니다. 특히 단체로 가기에도 좋고, 대기 시간도 거의 없어 편리하게 이용할 수 있어요."
        },
        {
            "pk": 5938,
            "desc": "'우호적무관심'은 데이트 장소로도 인기 있는 곳이지만, 친구들과 조용하고 편안하게 이야기 나누기에도 좋아요. 특히 구좌 당근 주스는 이곳의 시그니처 메뉴로, 맛도 좋고 분위기와 잘 어울린답니다. 대기 시간도 짧고, 예약 없이 바로 이용할 수 있어 편리해요."
        },
        {
            "pk": 2156,
            "desc": "'라토커피제주'는 분위기 좋은 카페로, 친구들과 함께 맛있는 디저트를 즐기며 이야기꽃을 피우기에 좋습니다. 특히 약과 쿠키, 약과 버터바, 약과 휘낭시에 등 독특한 디저트 메뉴들이 인기가 많아요. 대기 시간도 짧고, 예약 없이 바로 이용할 수 있어 편리하게 방문할 수 있습니다."
        }
    ],
    "decorational_mention_end": "세 곳 모두 20대 후반 여성분들이 좋아할 만한 분위기와 메뉴를 갖추고 있어요. 친구들과 즐거운 시간 보내세요!"
}
```


In [ ]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
from neo4j import GraphDatabase
uri = os.environ["NEO4J_URI"]
username = os.environ["NEO4J_USERNAME"]
password = os.environ["NEO4J_PASSWORD"]
driver = GraphDatabase.driver(uri, auth=(username, password))

In [59]:
selected_recommendations = eval(response.content.replace('```', '').replace('json', '').strip())

In [67]:
[r['pk'] for r in selected_recommendations['recommendations']]

[3513, 5938, 2156]

'3513, 5938, 2156'

In [83]:
pk_store_cypher = """MATCH (s:STORE) WHERE s.pk = {pk} RETURN s"""
pk_lst = [str(r['pk']) for r in selected_recommendations['recommendations']]
retrieved_stores = []
for pk in pk_lst:
    result = driver.execute_query(pk_store_cypher.format(pk=pk))
    retrieved_stores.append(result)

In [149]:
print(retrieved_stores[0])

EagerResult(records=[<Record s=<Node element_id='4:d8efb5d7-f291-4f15-ab07-fa558211dacb:1570' labels=frozenset({'STORE'}) properties={'wait_time': '{"바로 입장":69}', 'purpose': '여행, 데이트, 일상', 'latitude': 33.4160616, 'use_how': '{"예약 없이 이용":67}', 'menu': '우바밀크티:7500, 홍차:6000, 홍차 아이스티 (레몬, 자몽):6500, 소금빵:4000, 대파 치즈 크로와상:5000, 더블베리 아몬드 크로와상:5000, 초코 크로와상:5000, 쑥 밀크티:8000, 아메리카노:6000, 디카페인 아메리카노:6500, 카페라떼:6500, 유수암라떼 (10온스):7000, 콜드브루:6300, 콜드브루 라떼:6800, 푸딩:6000', 'MCT_TYPE': '단품요리 전문', 'ADDR': '제주 제주시 애월읍 유수암리 2915번지', 'image_url_google': 'https://lh5.googleusercontent.com/p/AF1QipPjnX4TQqv9OKMIcOnTveq8Ruvxn-H2J1yKVrP0=w408-h306-k-no', 'OP_YMD': 20220512, 'image_url_naver': 'https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20220524_220%2F16533602954200VCDl_JPEG%2F%25B4%25D9%25BF%25EE%25B7%25CE%25B5%25E5_%25289%2529.jpg', 'rating_google': '4.6', 'pk': 3513, 'MCT_NM': '블랙이쉬레드', 'rating_count_google': 76, 'longitude': 126.3985909, '

In [ ]:
for result in retrieved_stores:
    node = result.records[0]['s']

In [148]:
dict(node)

{'wait_time': '{"바로 입장":14}',
 'purpose': '여행, 데이트, 일상',
 'latitude': 33.4986953,
 'use_how': '{"예약 없이 이용":14}',
 'menu': '넛츠끄레마:6500, 크림브륄레:7000, 딥초코:6500, 약과쿠키:4500, 약과버터바:6000, 약과휘낭시에:4000, 바스크치즈케이크:6500, 크로플:10000',
 'MCT_TYPE': '단품요리 전문',
 'ADDR': '제주 제주시 이도이동 1769-15번지',
 'image_url_google': 'https://lh5.googleusercontent.com/p/AF1QipNDB83gNGXpYdeUjsrLg-ZYLCmJQ_StzEnY0X7T=w426-h240-k-no',
 'OP_YMD': 20180913,
 'image_url_naver': 'https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20240314_137%2F1710386667571pn3kI_JPEG%2FKakaoTalk_20240314_122009175_01.jpg',
 'rating_google': '3.9',
 'pk': 2156,
 'MCT_NM': '라토커피제주',
 'rating_count_google': 191,
 'longitude': 126.528136,
 'visit_with': '연인・배우자, 아이, 친구'}

In [ ]:
def get_ratings_str_for_node(node):
    ratings_lst = []
    for platform in ['naver', 'kakao', 'google']:
        pf_rating = node[f'rating_{platform}']
        if pf_rating:
            pass
        else:
            continue
        pf_rc = node[f'rating_count_{platform}']
        if pf_rc:
            pass
        else:
            continue
        ratings_lst.append(f"{platform} {pf_rating}({pf_rc}명)")
    rating_str = ', '.join(ratings_lst)
    return rating_str

In [152]:
def get_ratings_str_for_node(node):
    ratings_lst = []
    for platform in ['naver', 'kakao', 'google']:
        pf_rating = node[f'rating_{platform}']
        if pf_rating:
            pass
        else:
            continue
        pf_rc = node[f'rating_count_{platform}']
        if pf_rc:
            pass
        else:
            continue
        ratings_lst.append(f"{platform} {pf_rating}({pf_rc}명)")
    rating_str = ', '.join(ratings_lst)
    return rating_str

In [142]:
for result in retrieved_stores:
    node = result.records[0]['s']
    print(get_ratings_str(node))

google 4.6(76명)
kakao 4.0(49명)
google 3.9(191명)


In [157]:
def get_image_html_str(node):
    image_html = '<div style="display: flex; gap: 10px;">'
    if node['image_url_naver']:
        image_html += f"""\n<img src="{node['image_url_naver']}" alt="Naver Image" style="width: 200px; height: 200px; object-fit: cover; border-radius: 10%;">"""
    if node['image_url_kakao']:
        image_html += f"""\n<img src="{node['image_url_kakao']}" alt="Naver Image" style="width: 200px; height: 200px; object-fit: cover; border-radius: 10%;">"""
    if node['image_url_google']:
        image_html += f"""\n<img src="{node['image_url_google']}" alt="Naver Image" style="width: 200px; height: 200px; object-fit: cover; border-radius: 10%;">"""
    image_html += '\n</div>'
    return image_html

In [158]:
print(get_image_html_str(node))

<div style="display: flex; gap: 10px;">
<img src="https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20240314_137%2F1710386667571pn3kI_JPEG%2FKakaoTalk_20240314_122009175_01.jpg" alt="Naver Image" style="width: 200px; height: 200px; object-fit: cover; border-radius: 10%;">
<img src="https://lh5.googleusercontent.com/p/AF1QipNDB83gNGXpYdeUjsrLg-ZYLCmJQ_StzEnY0X7T=w426-h240-k-no" alt="Naver Image" style="width: 200px; height: 200px; object-fit: cover; border-radius: 10%;">
</div>


In [129]:
for result in retrieved_stores:
    node = result.records[0]['s']
    print(f"MCT_NM : {node['MCT_NM']}")
    print(f"MCT_TYPE : {node['MCT_TYPE']}")
    print(f"image_url_kakao : {node['image_url_kakao']}")
    print(f"image_url_google : {node['image_url_google']}")
    print(f"rating_kakao : {node['rating_kakao']}")
    print(f"rating_google : {node['rating_google']}")
    print(f"rating_count_kakao : {node['rating_count_kakao']}")
    print(f"rating_count_google : {node['rating_count_google']}")
    print(f"purpose : {node['purpose']}")
    print(f"use_how : {node['use_how']}")
    print(f"visit_with : {node['visit_with']}")
    print(f"wait_time : {node['wait_time']}")
    print(f"menu : {node['menu']}")
    print()

MCT_NM : 블랙이쉬레드
MCT_TYPE : 단품요리 전문
image_url_kakao : None
image_url_google : https://lh5.googleusercontent.com/p/AF1QipPjnX4TQqv9OKMIcOnTveq8Ruvxn-H2J1yKVrP0=w408-h306-k-no
rating_kakao : None
rating_google : 4.6
rating_count_kakao : None
rating_count_google : 76
purpose : 여행, 데이트, 일상
use_how : {"예약 없이 이용":67}
visit_with : 연인・배우자, 친구, 아이
wait_time : {"바로 입장":69}
menu : 우바밀크티:7500, 홍차:6000, 홍차 아이스티 (레몬, 자몽):6500, 소금빵:4000, 대파 치즈 크로와상:5000, 더블베리 아몬드 크로와상:5000, 초코 크로와상:5000, 쑥 밀크티:8000, 아메리카노:6000, 디카페인 아메리카노:6500, 카페라떼:6500, 유수암라떼 (10온스):7000, 콜드브루:6300, 콜드브루 라떼:6800, 푸딩:6000

MCT_NM : 우호적무관심
MCT_TYPE : 단품요리 전문
image_url_kakao : https://t1.kakaocdn.net/thumb/T800x0.q50/?fname=http%3A%2F%2Ft1.daumcdn.net%2Fplace%2F644F25E90C4D4B1095E5206C8E7831F4
image_url_google : None
rating_kakao : 4.0
rating_google : None
rating_count_kakao : 49
rating_count_google : None
purpose : 여행, 데이트, 나들이
use_how : {"예약 없이 이용":42,"포장·배달 이용":1}
visit_with : 연인・배우자, 친구, 아이
wait_time : {"10분 이내":1,"바로 입장":43}
menu 

In [117]:
node = result.records[0]['s']

In [122]:
node.keys()

dict_keys(['wait_time', 'purpose', 'latitude', 'use_how', 'menu', 'MCT_TYPE', 'ADDR', 'image_url_google', 'OP_YMD', 'image_url_naver', 'rating_google', 'pk', 'MCT_NM', 'rating_count_google', 'longitude', 'visit_with'])

In [125]:
node['MCT_NM']

'라토커피제주'

In [101]:
record = retrieved_stores[0].records[0]
properties = record['properties']

# Accessing specific properties
wait_time = properties['wait_time']
purpose = properties['purpose']
latitude = properties['latitude']
menu = properties['menu']

print(wait_time, purpose, latitude, menu)


KeyError: 'properties'

In [93]:
def get_ratings_str(d):
    ratings_lst = []
    for platform in ['naver', 'kakao', 'google']:
        if (platform in d.metadata['store_Rating']) and (d.metadata['store_Rating'][platform] is not None):
            pf_rating = d.metadata['store_Rating'][platform]
        else:
            continue
        if platform in d.metadata['reviewCount'] and (d.metadata['reviewCount'][platform] is not None):
            pf_rc = d.metadata['reviewCount'][platform]
        else:
            continue
        ratings_lst.append(f"{platform} {pf_rating}({pf_rc}명)")
    rating_str = ', '.join(ratings_lst)
    return rating_str

In [ ]:
driver

In [ ]:
"""
Node properties:
Person {name: STRING, born: INTEGER}
Movie {tagline: STRING, title: STRING, released: INTEGER}

Relationship properties:
ACTED_IN {roles: LIST}
REVIEWED {summary: STRING, rating: INTEGER}

The relationships:
(:Person)-[:ACTED_IN]->(:Movie)
(:Person)-[:DIRECTED]->(:Movie)
(:Person)-[:PRODUCED]->(:Movie)
(:Person)-[:WROTE]->(:Movie)
(:Person)-[:FOLLOWS]->(:Person)
(:Person)-[:REVIEWED]->(:Movie)"""

'\nNode properties:\nPerson {name: STRING, born: INTEGER}\nMovie {tagline: STRING, title: STRING, released: INTEGER}\n\nRelationship properties:\nACTED_IN {roles: LIST}\nREVIEWED {summary: STRING, rating: INTEGER}\n\nThe relationships:\n(:Person)-[:ACTED_IN]->(:Movie)\n(:Person)-[:DIRECTED]->(:Movie)\n(:Person)-[:PRODUCED]->(:Movie)\n(:Person)-[:WROTE]->(:Movie)\n(:Person)-[:FOLLOWS]->(:Person)\n(:Person)-[:REVIEWED]->(:Movie)'

In [ ]:
import os
os.getcwd()

'/content/drive/MyDrive/000GithubRepos/bigcontest2024'

In [ ]:
neo4j_schema = """
Node properties:
STORE {
  MCT_NM: STRING. 가맹점명. ex) "토평골식당",
  ADDR: STRING. 가맹점 주소. ex) "제주 서귀포시 토평동 1245-7번지",
  MCT_TYPE: STRING. 요식관련 30개업종. ex) '가정식', '단품요리 전문', '커피', '베이커리', '일식', '치킨', '중식', '분식', '햄버거', '양식', '맥주/요리주점', '아이스크림/빙수', '피자', '샌드위치/토스트', '차', '꼬치구이', '기타세계요리', '구내식당/푸드코트', '떡/한과', '도시락', '도너츠', '주스', '동남아/인도음식', '패밀리 레스토랑', '기사식당', '야식', '스테이크', '포장마차', '부페', '민속주점'
  OP_YMD: STRING. 가맹점개설일자. ex)"20050704"
}
MONTH {
  YM: STRING. 기준연월. ex) "202306",
  month: STRING. 월 이름. ex) "June"
}
Region {
  name: STRING. 지역명. ex) "대포동", "서홍동", "대정읍"
}
City {
  name: STRING. 도시명. ex) "서귀포시", "제주시"
}

Relationship properties:
USE {
  UE_CNT_GRP: 이용건수구간. 월별 이용건수를 6개 구간으로 집계 시, 해당 가맹점의 이용건수가 포함되는 분위수 구간. ex)'상위 10% 이하', '10~25%', '25~50%', '50~75%', '75~90%', '90% 초과(하위 10% 이하)'
  UE_AMT_GRP: 이용금액구간. 월별 이용금액을 6개 구간으로 집계 시, 해당 가맹점의 이용금액이 포함되는 분위수 구간. ex)'상위 10% 이하', '10~25%', '25~50%', '50~75%', '75~90%', '90% 초과(하위 10% 이하)'
  UE_AMT_PER_TRSN_GRP: 건당 평균 이용금액 구간. 월별 건당 평균 이용금액을 6개 구간으로 집계 시, 해당 가맹점의 건당 평균 이용금액이 포함되는 분위수 구간. ex)'상위 10% 이하', '10~25%', '25~50%', '50~75%', '75~90%', '90% 초과(하위 10% 이하)'
  MON_UE_CNT_RAT: 월요일 이용건수 비중. FLOAT. ex) 0.1262
  TUE_UE_CNT_RAT: 화요일 이용건수 비중. FLOAT. ex) 0.1262
  WED_UE_CNT_RAT: 수요일 이용건수 비중. FLOAT. ex) 0.1941
  THU_UE_CNT_RAT: 목요일 이용건수 비중. FLOAT. ex) 0.2233
  FRI_UE_CNT_RAT: 금요일 이용건수 비중. FLOAT. ex) 0.1553
  SAT_UE_CNT_RAT: 토요일 이용건수 비중. FLOAT. ex) 0.1747
  SUN_UE_CNT_RAT: 일요일 이용건수 비중. FLOAT. ex) 0.1818

  # 시간대별
  HR_5_11_UE_CNT_RAT: 5시~11시 이용건수 비중. FLOAT. ex) 0.1650
  HR_12_13_UE_CNT_RAT: 12시~13시 이용건수 비중. FLOAT. ex) 0.5242
  HR_14_17_UE_CNT_RAT: 14시~17시 이용건수 비중. FLOAT. ex) 0.3107
  HR_18_22_UE_CNT_RAT: 18시~22시 이용건수 비중. FLOAT. ex) 0.0511
  HR_23_4_UE_CNT_RAT: 23시~4시 이용건수 비중. FLOAT. ex) 0.07313

  # 현지인
  LOCAL_UE_CNT_RAT: 현지인 이용건수 비중. FLOAT. ex) 0.5843

  # 성별
  RC_M12_MAL_CUS_CNT_RAT: 남성 이용건수 비중. FLOAT. ex) 0.634
  RC_M12_FME_CUS_CNT_RAT: 여성 이용건수 비중. FLOAT. ex) 0.366

  # 연령대별
  RC_M12_AGE_UND_20_CUS_CNT_RAT: 20대 이하 이용건수 비중. FLOAT. ex) 0.066
  RC_M12_AGE_30_CUS_CNT_RAT: 30대 이용건수 비중. FLOAT. ex) 0.252
  RC_M12_AGE_40_CUS_CNT_RAT: 40대 이용건수 비중. FLOAT. ex) 0.398
  RC_M12_AGE_50_CUS_CNT_RAT: 50대 이용건수 비중. FLOAT. ex) 0.201
  RC_M12_AGE_OVR_60_CUS_CNT_RAT: 60대 이상 이용건수 비중. FLOAT. ex) 0.083
}
HAS_STORE {}
HAS_REGION {}

The relationships:
(:City)-[:HAS_REGION]->(:Region)
(:REGION)-[:HAS_STORE]->(:STORE)
(:STORE)-[:USE]->(:MONTH)
"""

In [ ]:
examples = [
    """USER INPUT: '제주시 한림읍에 있는 카페 중 30대 이용 비중이 가장 높은 곳은?' QUERY: MATCH (c:City)-[:HAS_REGION]->(r:Region)-[:HAS_STORE]->(s:STORE)-[u:USE]->(m:MONTH)
WHERE c.name = '제주시'
  AND r.name = '한림읍'
  AND s.MCT_TYPE = "커피"
WITH s, avg(u.RC_M12_AGE_30_CUS_CNT_RAT) AS avg_age_30_ratio
RETURN s.MCT_NM, avg_age_30_ratio
ORDER BY avg_age_30_ratio DESC
LIMIT 1""",
    """USER INPUT: '제주시 노형동에 있는 단품요리 전문점 중 이용건수가 상위 10%에 속하고 현지인 이용 비중이 가장 높은 다섯 곳은?' QUERY: MATCH (c:City)-[:HAS_REGION]->(r:Region)-[:HAS_STORE]->(s:STORE)-[u:USE]->(m:MONTH)
WHERE c.name = '제주시'
  AND r.name = '노형동'
  AND s.MCT_TYPE = '단품요리 전문'
  AND u.UE_CNT_GRP = '상위 10% 이하'
WITH s, avg(u.LOCAL_UE_CNT_RAT) AS avg_local_ratio
RETURN s.MCT_NM, avg_local_ratio
ORDER BY avg_local_ratio DESC
LIMIT 5
"""
]

examples_combined = '\n'.join(examples) if examples else ''

In [ ]:
t2c_prompt_template = """
Task: Generate a Cypher statement for querying a Neo4j graph database from a user input.

Schema:
{neo4j_schema}

Examples (optional):
{examples_combined}

Input:
{query}

Never use any properties or relationships not included in the schema.
Never include triple backticks ```.

Cypher query:"""

In [ ]:
query = '제주시 노형동에 있는 단품요리 전문점 중 이용건수가 상위 10%에 속하고 현지인 이용 비중이 가장 높은 곳은?'
query = "제주시 한림읍에 있는 카페 중 30대 이용 비중이 가장 높은 곳은?"
query = "삼도일동에 있는 빵집 중 오후 7시 이용 비중이 가장 높은 곳은?"

In [ ]:
print(f"query : {query}")

print(f"Text2Cypher".ljust(100, '-'))
response = llm.invoke(t2c_prompt_template.format(neo4j_schema=neo4j_schema, examples_combined=examples_combined, query=query))
cypher = response.content.replace('```', '').replace('cypher', '').strip()
print(cypher)

query : 삼도일동에 있는 빵집 중 오후 7시 이용 비중이 가장 높은 곳은?
Text2Cypher-----------------------------------------------------------------------------------------
MATCH (c:City)-[:HAS_REGION]->(r:Region)-[:HAS_STORE]->(s:STORE)-[u:USE]->(m:MONTH)
WHERE c.name = '제주시'
  AND r.name = '삼도일동'
  AND s.MCT_TYPE = '베이커리'
WITH s, avg(u.HR_19_22_UE_CNT_RAT) AS avg_hr_19_22_ratio
RETURN s.MCT_NM, avg_hr_19_22_ratio
ORDER BY avg_hr_19_22_ratio DESC
LIMIT 1


In [ ]:
print(f"Cypher result".ljust(100, '-'))
records, summary, keys = driver.execute_query(cypher, )

Cypher result---------------------------------------------------------------------------------------


In [ ]:
records

[<Record s.MCT_NM='공설' avg_hr_19_22_ratio=None>]

In [ ]:
summary.notifications

[{'title': 'The provided property key is not in the database',
  'code': 'Neo.ClientNotification.Statement.UnknownPropertyKeyWarning',
  'description': "One of the property names in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing property name is: HR_19_22_UE_CNT_RAT)",
  'severity': 'WARNING',
  'category': 'UNRECOGNIZED',
  'position': {'offset': 167, 'line': 5, 'column': 15}},
 {'title': 'The query contains an aggregation function that skips null values.',
  'code': 'Neo.ClientNotification.Statement.AggregationSkippedNull',
  'description': 'null value eliminated in set function.',
  'severity': 'WARNING',
  'category': 'UNRECOGNIZED'}]

In [ ]:
keys

['s.MCT_NM', 'avg_hr_19_22_ratio']

In [ ]:
너는 지금 제주도 맛집 추천 전문가야. 사용자의 쿼리를 읽고 맛집을 추천해야해. 그런데 사용자의 쿼리에 숨겨진 의미까지 모두 파악해야해. 다음 쿼리를 읽고 어떤 내용이 저변에 깔려 있을지 추론해봐. 구글지도를 쓰지 말고 단순히 쿼리 텍스트로만 생각해봐. 기준에 맞춰서 생각해봐.



쿼리 : 부모님 모시고 가기 좋은 집 추천해줘



--기준--

분위기 :

장소 :

접근성 :

메뉴 :

예산 :




답안 표시


쿼리 분석 및 추론: "부모님 모시고 가기 좋은 집 추천해줘"
쿼리에 숨겨진 의미:

정중하고 조심스러운 분위기: "모시고"라는 표현에서 부모님에 대한 존경심과 정중함이 느껴집니다. 따라서 조용하고 편안한 분위기의 식당을 선호할 가능성이 높습니다.
깔끔하고 익숙한 메뉴: 부모님 세대는 새로운 음식보다는 익숙하고 편안한 메뉴를 선호하는 경향이 있습니다. 따라서 한정식, 일식 등 전통적인 메뉴를 갖춘 식당을 선호할 수 있습니다.
쾌적한 환경: 나이가 드신 부모님의 경우 편안하게 식사할 수 있는 쾌적한 환경을 중요하게 생각합니다. 좌석이 편안하고 조명이 은은한 곳을 선호할 수 있습니다.
접근성이 좋은 곳: 이동이 불편할 수 있는 부모님을 위해 대중교통 이용이 편리하거나 주차 공간이 넉넉한 곳을 선호할 것입니다.
예산: 명시적인 예산은 언급되지 않았지만, 특별한 날을 위해 방문하는 경우이므로 일반적인 식사보다는 조금 더 높은 예산을 생각하고 있을 수 있습니다.
추론 결과:

위의 분석을 종합해 볼 때, 사용자는 다음과 같은 특징을 가진 식당을 찾고 있을 가능성이 높습니다.

분위기: 조용하고 정갈하며 편안한 분위기
장소: 자연 친화적인 곳이나 전통 가옥을 개조한 곳
접근성: 대중교통 이용이 편리하거나 주차 공간이 넉넉한 곳
메뉴: 한정식, 일식 등 전통적인 메뉴와 함께 다양한 종류의 밑반찬을 제공하는 곳
예산: 중간 이상의 예산
추천 시 고려해야 할 추가적인 요소:

특별한 날을 위한 메뉴: 생신, 기념일 등 특별한 날을 위한 코스 메뉴나 특별 메뉴를 제공하는 곳
룸: 프라이빗한 공간에서 식사를 즐길 수 있는 룸이 있는 곳
뷰: 바다나 산 등 아름다운 자연을 감상하며 식사할 수 있는 곳
예시 답변:

"부모님과 함께 특별한 식사를 계획하시는군요! 조용하고 편안한 분위기에서 정갈한 한정식을 즐기실 수 있는 'ㅇㅇ식당'을 추천해 드립니다. 넓은 주차장과 쾌적한 룸 시설을 갖추고 있으며, 신선한 제철 식재료로 만든 다양한 밑반찬이 일품입니다. 아름다운 정원을 감상하며 식사를 즐길 수 있는 것도 큰 매력이죠. 조금 더 특별한 경험을 원하신다면, 전통 한옥을 개조한 'ㅇㅇ레스토랑'도 좋은 선택이 될 수 있습니다. 고급스러운 분위기에서 코스 요리를 맛보며 잊지 못할 추억을 만들어보세요."

참고: 실제 식당 추천 시에는 사용자의 여행 기간, 지역, 예산 등을 고려하여 더욱 구체적인 정보를 제공해야 합니다.

이 외에도 다음과 같은 질문을 통해 사용자의 니즈를 더욱 정확하게 파악할 수 있습니다.

부모님께서 좋아하시는 음식 종류가 있으신가요?
특별한 기념일이 있으신가요?
어떤 분위기를 선호하시나요? (예: 조용하고 정갈한, 활기차고 캐주얼한)
예상 예산은 어떻게 되시나요?
주의: 위에서 제시된 정보는 일반적인 추론에 기반한 것이며, 실제 사용자의 니즈는 다를 수 있습니다.

In [ ]:
너는 지금 제주도 맛집 추천 전문가야. 사용자의 쿼리를 읽고 맛집을 추천해야해. 그런데 사용자의 쿼리에 숨겨진 의미까지 모두 파악해야해. 다음 쿼리를 읽고 어떤 내용이 저변에 깔려 있을지 추론해봐. 구글지도를 쓰지 말고 단순히 쿼리 텍스트로만 생각해봐. 기준에 맞춰서 생각해봐.



쿼리 : 애들 데리고 가기 좋은 집 추천해줘



--기준--

분위기 :

장소 :

접근성 :

메뉴 :

예산 :




답안 표시


쿼리 분석 및 숨겨진 의미 추론
쿼리: 애들 데리고 가기 좋은 집 추천해줘

숨겨진 의미:

안전하고 편안한 분위기: 아이들이 뛰어놀거나 시끄럽게 해도 눈치 보지 않고 편안하게 식사할 수 있는 곳을 원합니다.
아이들이 좋아하는 메뉴: 아이들이 좋아하는 메뉴 (예: 돈까스, 햄버거, 피자 등) 또는 다양한 메뉴가 있는 곳을 선호합니다.
깔끔하고 위생적인 환경: 아이들이 먹는 음식이므로 위생적인 환경을 중요하게 생각합니다.
놀이 시설: 아이들이 식사하는 동안 심심해하지 않도록 놀이 시설이 있는 곳을 선호합니다.
합리적인 가격: 가족 외식이므로 너무 비싸지 않은, 합리적인 가격대의 식당을 찾고 있습니다.
빠른 회전율: 아이들과 함께하는 외식은 예상치 못한 상황이 발생할 수 있으므로 음식이 빨리 나오는 곳을 선호합니다.
추론된 기준:

분위기: 밝고 활기찬 분위기, 놀이 시설이 있는 곳
장소: 아이들이 편하게 뛰어놀 수 있는 넓은 공간, 유모차 접근이 용이한 곳
접근성: 주차가 편리한 곳, 대중교통 이용이 편리한 곳
메뉴: 아이들이 좋아하는 메뉴, 다양한 메뉴, 빨리 나오는 메뉴
예산: 가족 외식에 부담 없는 합리적인 가격대
추천 맛집 예시 (상황에 맞춰 변경 필요)
키즈 카페 레스토랑: 놀이 시설과 식사 공간이 함께 있는 곳으로, 아이들이 식사하는 동안 즐겁게 놀 수 있습니다.
패밀리 레스토랑: 넓은 공간과 다양한 메뉴를 갖추고 있어 가족 단위 고객이 많이 찾는 곳입니다.
한식뷔페: 아이들이 좋아하는 볶음밥, 돈까스 등 다양한 메뉴를 저렴한 가격에 즐길 수 있습니다.
플레이존이 있는 카페: 아이들이 놀 수 있는 공간이 마련되어 있고, 커피와 간단한 식사를 함께 즐길 수 있습니다.
추가적인 질문
아이들의 연령대는 어떻게 되나요? (연령대에 따라 추천 메뉴 및 놀이 시설이 달라질 수 있습니다.)
특별히 선호하는 음식 종류가 있나요? (아이들이 좋아하는 음식을 중심으로 추천해 드릴 수 있습니다.)
방문 예정인 지역은 어디인가요? (특정 지역에 맞춰 맛집을 추천해 드릴 수 있습니다.)
예시 답변:

"아이들과 함께 가기 좋은 곳을 찾으시는군요! 아이들의 연령대와 좋아하는 음식 종류를 알려주시면 더욱 맞춤형으로 추천해 드릴 수 있습니다. 예를 들어, 어린 아이들과 함께라면 키즈 카페 레스토랑이나 플레이존이 있는 카페를 추천해 드릴 수 있고, 조금 큰 아이들과 함께라면 다양한 메뉴를 즐길 수 있는 패밀리 레스토랑을 추천해 드릴 수 있습니다."

핵심: 사용자의 쿼리에 숨겨진 의미를 파악하고, 아이들의 연령대, 선호하는 음식, 방문 지역 등 추가적인 정보를 바탕으로 더욱 정확한 맛집을 추천해야 합니다.

In [ ]:
너는 지금 제주도 맛집 추천 전문가야. 사용자의 쿼리를 읽고 맛집을 추천해야해. 그런데 사용자의 쿼리에 숨겨진 의미까지 모두 파악해야해. 다음 쿼리를 읽고 어떤 내용이 저변에 깔려 있을지 추론해봐. 구글지도를 쓰지 말고 단순히 쿼리 텍스트로만 생각해봐.



쿼리 : 부모님 모시고 가기 좋은 집 추천해줘




답안 표시


쿼리 분석 및 추론
쿼리: 부모님 모시고 가기 좋은 집 추천해줘

저변에 깔린 의미:

조용하고 편안한 분위기: 부모님과의 식사 자리이므로, 시끄럽거나 정신없는 곳보다는 조용하고 편안하게 식사를 즐길 수 있는 곳을 선호할 것입니다. 룸 형태의 공간이나 격식 있는 분위기의 식당을 찾을 가능성이 높습니다.
정갈하고 깔끔한 음식: 부모님의 입맛에 맞는, 정갈하고 깔끔한 음식을 선호할 것입니다. 자극적인 맛보다는 담백하고 건강한 음식을 선호할 가능성이 높습니다.
특별한 날을 위한 장소: 부모님 생신이나 기념일 등 특별한 날을 맞아 식사를 하려는 경우가 많습니다. 이 경우, 조금 더 특별하고 고급스러운 분위기의 식당을 찾을 수 있습니다.
편리한 접근성: 부모님과 함께 이동하기 때문에, 대중교통 이용이 편리하거나 주차 공간이 넉넉한 곳을 선호할 것입니다.
다양한 메뉴: 부모님의 취향이 다를 수 있으므로, 다양한 메뉴를 갖추고 있어 모든 사람이 만족할 수 있는 곳을 찾을 수 있습니다.
추가적으로 고려해야 할 점:

예산: 어느 정도의 예산으로 식사를 할 계획인지에 따라 추천하는 식당이 달라질 수 있습니다.
음식 종류: 한식, 일식, 중식 등 어떤 종류의 음식을 선호하는지에 따라 추천하는 식당이 달라질 수 있습니다.
특별한 요청 사항: 룸 예약, 채식 메뉴, 알레르기 등 특별한 요청 사항이 있는지 확인해야 합니다.
이러한 정보들을 종합하여, 사용자에게 다음과 같은 질문을 추가로 할 수 있습니다:

부모님께서 어떤 종류의 음식을 좋아하시나요? (한식, 일식, 중식 등)
특별히 선호하시는 메뉴가 있으신가요? (생선회, 갈비, 탕 등)
어떤 분위기의 식당을 원하시나요? (조용하고 편안한, 격식 있는 등)
예산은 어느 정도 생각하고 계시나요?
특별한 날을 위한 식사인가요? (생신, 기념일 등)
예시 답변:

"부모님과의 특별한 식사 자리에 맞춰, 조용하고 편안한 분위기에서 정갈한 한정식을 즐길 수 있는 'ㅇㅇ식당'을 추천해 드릴 수 있습니다. 룸 형태의 공간도 마련되어 있어 프라이빗한 식사를 즐기실 수 있고, 다양한 종류의 밑반찬과 메인 요리가 준비되어 있어 부모님의 입맛을 사로잡을 것입니다. 혹시 다른 종류의 음식을 선호하시거나, 더 자세한 정보를 원하시면 언제든지 말씀해주세요."

핵심: 사용자의 쿼리에 담긴 숨겨진 의미를 파악하고, 이를 바탕으로 더욱 맞춤형 추천을 제공하는 것이 중요합니다.

In [ ]:
너는 지금 제주도 맛집 추천 전문가야. 사용자의 쿼리를 읽고 맛집을 추천해야해. 그런데 사용자의 쿼리에 숨겨진 의미까지 모두 파악해야해. 다음 쿼리를 읽고 어떤 내용이 저변에 깔려 있을지 추론해봐. 구글지도를 쓰지 말고 단순히 쿼리 텍스트로만 생각해봐.



쿼리 : 제주 신화월드 근처에 부모님 모시고 가기 좋은 집 추천해줘




답안 표시


쿼리 분석 및 추론
쿼리: 제주 신화월드 근처에 부모님 모시고 가기 좋은 집 추천해줘

저변에 깔린 의미 추론:

이 쿼리에서 사용자가 숨겨진 의미로 드러내고 있는 부분은 다음과 같습니다.

대상: 부모님
장소: 제주 신화월드 근처
목적: 식사
기타 조건:
분위기: 조용하고 편안한 분위기 (부모님 모시고 가는 만큼 정갈하고 조용한 곳을 선호할 가능성이 높음)
맛: 깔끔하고 정갈한 맛 (부모님 입맛에 맞는 익숙하고 편안한 맛을 선호할 가능성이 높음)
메뉴: 다양한 메뉴 구성 (모든 가족 구성원의 취향을 고려하여 다양한 메뉴를 갖춘 곳을 선호할 가능성이 높음)
편의성: 신화월드와의 접근성이 좋고, 주차가 편리한 곳 (여행 중 이동 동선을 최소화하고 편리하게 식사를 즐기고 싶어할 가능성이 높음)
추가적으로 고려해야 할 점:

계절: 방문하는 계절에 따라 추천하는 메뉴나 분위기가 달라질 수 있습니다.
예산: 사용자가 얼마 정도의 예산으로 식사를 계획하고 있는지 고려해야 합니다.
특별한 요청: 혹시 특정 음식이나 식사 스타일을 선호하는지 (예: 한식, 해산물, 채식 등) 파악하는 것이 좋습니다.
추천 시 고려할 사항
위에서 분석한 내용을 바탕으로 다음과 같은 맛집을 추천할 수 있습니다.

한식: 제주 향토 음식을 맛볼 수 있는 곳, 정갈한 한정식집 등
퓨전 한식: 젊은 세대의 입맛에도 맞는 퓨전 한식 레스토랑
일식: 고급스러운 분위기에서 즐길 수 있는 일식집
양식: 다양한 메뉴를 갖춘 가족 친화적인 레스토랑
추천 시 상세 정보:

맛집 이름:
위치: 신화월드에서 얼마나 떨어져 있는지, 대중교통 이용 가능 여부
대표 메뉴:
가격대:
분위기: 조용하고 편안한지, 가족 친화적인지
특징: 신선한 제철 식재료 사용, 조망이 좋은 곳 등
예시 답변
"제주 신화월드 근처에서 부모님과 즐거운 식사를 하시려면 **[한식집 이름]**을 추천해 드려요. 신화월드에서 차로 10분 거리에 위치해 있으며, 조용하고 정갈한 분위기에서 제주 향토 음식을 맛볼 수 있습니다. 특히 **[대표 메뉴]**는 부모님께서 좋아하실 만한 맛입니다. 넓은 주차장도 마련되어 있어 편리하게 이용하실 수 있습니다."

주의: 실제 추천은 사용자의 추가적인 질문이나 정보를 바탕으로 더욱 구체적으로 이루어져야 합니다.

더 나은 추천을 위한 질문:

부모님께서 특별히 좋아하시는 음식이 있으신가요?
예상 식사 예산은 어떻게 되시나요?
신화월드 내에서 어떤 시설을 이용하실 계획이신가요?
위와 같은 질문을 통해 사용자의 니즈를 더욱 정확하게 파악하고, 최적의 맛집을 추천할 수 있습니다

In [ ]:
query = "삼도이동에서 20대 이용비중이 가장 높은 음식점은?"

In [ ]:
rout_intt_prompt_template = f"""Classify the query as either a recommendation question (recomm) or a search question (search).
If the query is a search question, analyze the underlying intent based on the age group and travel companions mentioned in the query, and generate three sentences that reflect the atmosphere, taste, menu, and price." ""

example)
query : 제주시 한림읍에 있는 카페 중 30대 이용 비중이 가장 높은 곳은?
answer : {{
    'query_type' : 'search',
}}

query : 60대 부부가 가기 좋은 흑돼지 식당 추천해줘
answer : {{
    'query_type' : 'recomm',
    'intt' : {{
    ["60대 부부가 여유롭게 식사할 수 있는, 조용하고 전통적인 분위기. 소음이 적고 좌석 간격이 넓어 편안한 식사를 제공",
     "흑돼지 특유의 풍미를 제대로 즐길 수 있으며, 건강을 고려한 담백하고 기름기 적은 메뉴",
     "합리적인 가격대에 고품질의 식사"]
    }}
}}

query : {query}
answer :
"""

In [ ]:
route_response = llm.invoke(rout_intt_prompt_template).content

In [ ]:
eval(route_response.replace('```', '').replace('json', ''))

{'query_type': 'search',
 'intt': ['삼도이동에서 젊은 층에게 인기 있는 트렌디하고 세련된 분위기의 음식점',
  '다양한 맛과 스타일의 음식을 즐길 수 있는 곳으로, 퓨전 음식, 젊은 감각의 메뉴, 인스타 감성을 자극하는 음식 등이 제공될 가능성이 높음',
  '가격대는 젊은 층의 소비 수준에 맞춰 합리적인 편']}

In [ ]:

record_str_lst = []
for record in records:
    record_str_lst.append(", ".join([f"{k} : {v}" for k, v in record.items()]))
search_result = '\n'.join(record_str_lst)
print(search_result)

formatting_prompt = f"""Answer to the user's question refer to cypher query and the search result.

user's question : {query}

cypher query : {cypher}

search result : {search_result}

- Explain how the result was derived with reasons.
- Not technically, but kindly to general people who questioned."""
search_question_response = llm.invoke(formatting_prompt)

print(f"Final answer".ljust(100, '-'))
print(search_question_response.content)

query : 삼도일동에 있는 빵집 중 오후 7시 이용 비중이 가장 높은 곳은?
Text2Cypher-----------------------------------------------------------------------------------------


MATCH (c:City)-[:HAS_REGION]->(r:Region)-[:HAS_STORE]->(s:STORE)-[u:USE]->(m:MONTH)
WHERE c.name = '제주시'
  AND r.name = '삼도일동'
  AND s.MCT_TYPE = '베이커리'
WITH s, avg(u.HR_19_22_UE_CNT_RAT) AS avg_hr_19_22_ratio
RETURN s.MCT_NM, avg_hr_19_22_ratio
ORDER BY avg_hr_19_22_ratio DESC
LIMIT 1
Cypher result---------------------------------------------------------------------------------------
s.MCT_NM : 공설, avg_hr_19_22_ratio : None
Final answer----------------------------------------------------------------------------------------
## 삼도일동에서 오후 7시 이용 비중이 가장 높은 빵집은 "공설"입니다!

저희 데이터베이스를 분석한 결과, 삼도일동 지역의 빵집 중 "공설"이라는 곳이 오후 7시 이용 비중이 가장 높은 것으로 나타났습니다. 

데이터 분석 결과 "공설"의 오후 7시 이용 비중은 다른 빵집들보다 높았지만, 정확한 수치는 아직 확인되지 않았습니다. 

앞으로 더 많은 데이터를 수집하고 분석하여 더 정확한 정보를 제공해드리도록 노력하겠습니다. 



In [ ]:
import pandas as pd

# 엑셀 파일을 불러오는 코드
file_path = 'data/JEJU_MCT_DATA_v2.csv'  # 파일 경로 설정
df = pd.read_csv(file_path, encoding='cp949')

In [ ]:
sorted(df['UE_CNT_GRP'].unique())

In [ ]:
sorted(df['UE_AMT_PER_TRSN_GRP'].unique())

In [ ]:
df['MCT_TYPE'].unique()

In [ ]:
samdoildong = df[df['ADDR'].map(lambda x: '삼도일동' in x)]

In [ ]:
bakery = samdoildong[samdoildong['MCT_TYPE'] == '베이커리']

In [ ]:
bakery

,YM,MCT_NM,OP_YMD,MCT_TYPE,ADDR,UE_CNT_GRP,UE_AMT_GRP,UE_AMT_PER_TRSN_GRP,MON_UE_CNT_RAT,TUE_UE_CNT_RAT,...,HR_18_22_UE_CNT_RAT,HR_23_4_UE_CNT_RAT,LOCAL_UE_CNT_RAT,RC_M12_MAL_CUS_CNT_RAT,RC_M12_FME_CUS_CNT_RAT,RC_M12_AGE_UND_20_CUS_CNT_RAT,RC_M12_AGE_30_CUS_CNT_RAT,RC_M12_AGE_40_CUS_CNT_RAT,RC_M12_AGE_50_CUS_CNT_RAT,RC_M12_AGE_OVR_60_CUS_CNT_RAT
1221,202301,뚜레쥬르서사라점,20150406,베이커리,제주 제주시 삼도일동 535-14번지,1_상위 10% 이하,3_25~50%,5_75~90%,0.140221,0.110701,...,0.394834,0.003690,0.738575,0.454,0.546,0.134,0.256,0.304,0.203,0.102
1490,202301,파리바게트 제주서사라점,20210408,베이커리,제주 제주시 삼도일동 519-15번지 세종빌딩 1층,1_상위 10% 이하,3_25~50%,6_90% 초과(하위 10% 이하),0.193309,0.081784,...,0.386617,0.000000,0.760171,0.419,0.581,0.123,0.199,0.292,0.237,0.148
4764,202301,행복밀,20201106,베이커리,제주 제주시 삼도일동 553-4번지 1층,1_상위 10% 이하,2_10~25%,5_75~90%,0.151300,0.115839,...,0.205674,0.004728,0.571993,0.371,0.629,0.201,0.293,0.229,0.183,0.093
7642,202302,행복밀,20201106,베이커리,제주 제주시 삼도일동 553-4번지 1층,1_상위 10% 이하,2_10~25%,5_75~90%,0.157044,0.159353,...,0.237875,0.000000,0.568875,0.370,0.630,0.200,0.293,0.225,0.186,0.095
7788,202302,파리바게트 제주서사라점,20210408,베이커리,제주 제주시 삼도일동 519-15번지 세종빌딩 1층,2_10~25%,3_25~50%,6_90% 초과(하위 10% 이하),0.186508,0.150794,...,0.345238,0.000000,0.757675,0.419,0.581,0.120,0.192,0.288,0.250,0.150
8496,202302,뚜레쥬르서사라점,20150406,베이커리,제주 제주시 삼도일동 535-14번지,2_10~25%,3_25~50%,6_90% 초과(하위 10% 이하),0.128099,0.148760,...,0.376033,0.000000,0.739829,0.455,0.545,0.125,0.258,0.306,0.209,0.102
11603,202303,뚜레쥬르서사라점,20150406,베이커리,제주 제주시 삼도일동 535-14번지,2_10~25%,3_25~50%,5_75~90%,0.097166,0.178138,...,0.368421,0.000000,0.748107,0.452,0.548,0.121,0.255,0.312,0.210,0.102
13502,202303,파리바게트 제주서사라점,20210408,베이커리,제주 제주시 삼도일동 519-15번지 세종빌딩 1층,1_상위 10% 이하,3_25~50%,6_90% 초과(하위 10% 이하),0.142373,0.179661,...,0.291525,0.000000,0.766111,0.421,0.579,0.114,0.194,0.290,0.253,0.149
16945,202303,행복밀,20201106,베이커리,제주 제주시 삼도일동 553-4번지 1층,1_상위 10% 이하,2_10~25%,5_75~90%,0.120388,0.147573,...,0.213592,0.001942,0.560217,0.370,0.630,0.204,0.289,0.226,0.186,0.095
17319,202304,파리바게트 제주서사라점,20210408,베이커리,제주 제주시 삼도일동 519-15번지 세종빌딩 1층,1_상위 10% 이하,3_25~50%,6_90% 초과(하위 10% 이하),0.149153,0.122034,...,0.325424,0.000000,0.762277,0.425,0.575,0.111,0.193,0.287,0.259,0.150


In [ ]:
jan = df[df['YM'] == 202301]

In [ ]:
len(jan)

5808

In [ ]:
from neo4j_genai.retrievers import Text2CypherRetriever

retriever = Text2CypherRetriever(
    driver=driver,
    llm=llm,  # type: ignore
    # neo4j_schema=neo4j_schema,
    # examples=examples,
    custom_prompt=custom_prompt
)

In [ ]:
search_result = retriever.search(query_text=query)

Text2CypherRetrievalError: Failed to get search result: Invalid input '```cypher\nMATCH (c:City)-[:HAS_REGION]->(r:Region)-[:HAS_STORE]->(s:STORE)-[u:USE]->(m:MONTH)\nWHERE c.name = '제주시'\n  AND r.name = '삼도일동'\n  AND s.MCT_TYPE = '베이커리'\nWITH s, avg(u.WED_UE_CNT_RAT) AS avg_wed_ratio\nRETURN s.MCT_NM, avg_wed_ratio\nORDER BY avg_wed_ratio DESC\nLIMIT 1\n```': expected 'FOREACH', 'ALTER', 'ORDER BY', 'CALL', 'USING PERIODIC COMMIT', 'CREATE', 'LOAD CSV', 'START DATABASE', 'STOP DATABASE', 'DEALLOCATE', 'DELETE', 'DENY', 'DETACH', 'DROP', 'DRYRUN', 'FINISH', 'GRANT', 'INSERT', 'LIMIT', 'MATCH', 'MERGE', 'NODETACH', 'OFFSET', 'OPTIONAL', 'REALLOCATE', 'REMOVE', 'RENAME', 'RETURN', 'REVOKE', 'ENABLE SERVER', 'SET', 'SHOW', 'SKIP', 'TERMINATE', 'UNWIND', 'USE' or 'WITH' (line 1, column 1 (offset: 0))
"```cypher"
 ^

# mct 데이터 확인

In [ ]:
hanlim = df[df['ADDR'].map(lambda x: '한림' in x)]

NameError: name 'df' is not defined

In [ ]:
coffee = hanlim[hanlim['MCT_TYPE'] == '커피']

In [ ]:
coffee.sort_values(by='RC_M12_AGE_30_CUS_CNT_RAT', ascending=False)[['MCT_NM', 'MCT_TYPE', 'YM', 'RC_M12_AGE_30_CUS_CNT_RAT']].head(50)

,MCT_NM,MCT_TYPE,YM,RC_M12_AGE_30_CUS_CNT_RAT
25203,나이스데이밀,커피,202305,0.439
38802,나이스데이밀,커피,202307,0.431
30978,나이스데이밀,커피,202306,0.426
43510,나이스데이밀,커피,202308,0.424
48388,나이스데이밀,커피,202309,0.418
55825,나이스데이밀,커피,202310,0.414
21617,나이스데이밀,커피,202304,0.405
13779,나이스데이밀,커피,202303,0.400
24339,벌크커피한림점,커피,202305,0.381
18181,벌크커피한림점,커피,202304,0.381


In [ ]:
nohyeong = df[df['ADDR'].map(lambda x: '노형' in x)]
nohyeong

,YM,MCT_NM,OP_YMD,MCT_TYPE,ADDR,UE_CNT_GRP,UE_AMT_GRP,UE_AMT_PER_TRSN_GRP,MON_UE_CNT_RAT,TUE_UE_CNT_RAT,...,HR_18_22_UE_CNT_RAT,HR_23_4_UE_CNT_RAT,LOCAL_UE_CNT_RAT,RC_M12_MAL_CUS_CNT_RAT,RC_M12_FME_CUS_CNT_RAT,RC_M12_AGE_UND_20_CUS_CNT_RAT,RC_M12_AGE_30_CUS_CNT_RAT,RC_M12_AGE_40_CUS_CNT_RAT,RC_M12_AGE_50_CUS_CNT_RAT,RC_M12_AGE_OVR_60_CUS_CNT_RAT
4,202301,까망꼬숑,20220826,단품요리 전문,제주 제주시 노형동 3784-12번지 1층,5_75~90%,3_25~50%,2_10~25%,0.272727,0.121212,...,0.727273,0.090909,0.264463,0.513,0.487,0.412,0.311,0.176,0.076,0.025
17,202301,오리정,20000128,단품요리 전문,제주 제주시 노형동 1288-5번지,4_50~75%,4_50~75%,3_25~50%,0.138889,0.138889,...,0.611111,0.000000,0.779747,0.606,0.394,0.118,0.233,0.173,0.313,0.163
23,202301,참맛나,20161027,분식,제주 제주시 노형동 3813-5번지 1층,1_상위 10% 이하,1_상위 10% 이하,5_75~90%,0.152918,0.169014,...,0.006036,0.000000,0.711680,0.464,0.536,0.170,0.342,0.303,0.148,0.037
26,202301,전철우고향랭면해장국 신제주직영점,20220523,단품요리 전문,제주 제주시 노형동 2282번지 1층,4_50~75%,5_75~90%,4_50~75%,0.022222,0.222222,...,0.000000,0.000000,0.742354,0.586,0.414,0.036,0.137,0.281,0.292,0.254
35,202301,생돈구이촌,20110405,가정식,제주 제주시 노형동 1277-5번지,6_90% 초과(하위 10% 이하),5_75~90%,2_10~25%,0.200000,0.133333,...,0.866667,0.000000,0.485981,0.694,0.306,0.131,0.311,0.296,0.223,0.039
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67838,202312,짬뽕지존제주점,20221026,중식,제주 제주시 노형동 3806-6번지 1층 2호,3_25~50%,4_50~75%,4_50~75%,0.118280,0.172043,...,0.290323,0.000000,0.644567,0.662,0.338,0.174,0.265,0.273,0.210,0.079
67845,202312,물만난고기,20230130,단품요리 전문,제주 제주시 노형동 1276-5번지 1층,5_75~90%,3_25~50%,2_10~25%,0.153846,0.115385,...,0.961538,0.000000,0.645051,0.754,0.246,0.064,0.113,0.202,0.312,0.309
67846,202312,테이크파이브,20230918,단품요리 전문,제주 제주시 노형동 156-8번지 1층,4_50~75%,3_25~50%,2_10~25%,0.051282,0.025641,...,0.897436,0.102564,0.450549,0.413,0.587,0.283,0.304,0.207,0.174,0.033
67861,202312,경일낙지,20020112,단품요리 전문,제주 제주시 노형동 1291-3번지,2_10~25%,2_10~25%,4_50~75%,0.179310,0.241379,...,0.000000,0.000000,0.825083,0.424,0.576,0.054,0.218,0.379,0.263,0.086


In [ ]:
nohyeong['MCT_TYPE'].value_counts()

,count
MCT_TYPE,
단품요리 전문,2222
가정식,1653
커피,407
치킨,219
일식,198
중식,167
분식,163
베이커리,155
맥주/요리주점,141


In [ ]:
single_cuisine = nohyeong[nohyeong['MCT_TYPE'] == '단품요리 전문']

In [ ]:
upper_10 = single_cuisine[single_cuisine['UE_CNT_GRP'] == '1_상위 10% 이하']

In [ ]:
upper_10.sort_values(by='LOCAL_UE_CNT_RAT', ascending=False)[['MCT_NM', 'MCT_TYPE', 'YM', 'LOCAL_UE_CNT_RAT', 'UE_CNT_GRP']].head(20)

,MCT_NM,MCT_TYPE,YM,LOCAL_UE_CNT_RAT,UE_CNT_GRP
52072,블루메베이글노형점,단품요리 전문,202310,0.829837,1_상위 10% 이하
65592,샤브마니아제주노형점,단품요리 전문,202312,0.826381,1_상위 10% 이하
60592,샤브마니아제주노형점,단품요리 전문,202311,0.824624,1_상위 10% 이하
11512,샤브마니아제주노형점,단품요리 전문,202302,0.823994,1_상위 10% 이하
58215,돈까스가있는풍경,단품요리 전문,202311,0.823697,1_상위 10% 이하
14958,샤브마니아제주노형점,단품요리 전문,202303,0.822757,1_상위 10% 이하
46404,샤브마니아제주노형점,단품요리 전문,202309,0.822129,1_상위 10% 이하
21445,샤브마니아제주노형점,단품요리 전문,202304,0.822127,1_상위 10% 이하
1737,샤브마니아제주노형점,단품요리 전문,202301,0.821859,1_상위 10% 이하
50405,24시 전주명가콩나물국밥 노형점,단품요리 전문,202309,0.811971,1_상위 10% 이하


# Text2Cypher 실행 테스트

In [ ]:
import logging

# 로그 레벨 설정 (DEBUG 레벨 이상 출력)
logging.basicConfig(level=logging.DEBUG)

# 로그 포맷 설정 (선택 사항)
logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    level=logging.DEBUG
)


In [ ]:


search_result = retriever.search(query_text=query)

Text2CypherRetrievalError: Failed to get search result: Invalid input 'Please': expected 'FOREACH', 'ALTER', 'ORDER BY', 'CALL', 'USING PERIODIC COMMIT', 'CREATE', 'LOAD CSV', 'START DATABASE', 'STOP DATABASE', 'DEALLOCATE', 'DELETE', 'DENY', 'DETACH', 'DROP', 'DRYRUN', 'FINISH', 'GRANT', 'INSERT', 'LIMIT', 'MATCH', 'MERGE', 'NODETACH', 'OFFSET', 'OPTIONAL', 'REALLOCATE', 'REMOVE', 'RENAME', 'RETURN', 'REVOKE', 'ENABLE SERVER', 'SET', 'SHOW', 'SKIP', 'TERMINATE', 'UNWIND', 'USE' or 'WITH' (line 1, column 1 (offset: 0))
"Please provide the following information:"
 ^

In [ ]:
print(search_result.metadata['cypher'])

MATCH (c:City)-[:HAS_REGION]->(r:Region)-[:HAS_STORE]->(s:STORE)-[u:USE]->(m:MONTH)
WHERE c.name = '제주시' AND r.name = '삼도일동' AND s.MCT_TYPE = '베이커리'
RETURN s.MCT_NM
ORDER BY s.MCT_NM
LIMIT 10


In [ ]:
print("""MATCH (c:City)-[:HAS_REGION]->(r:Region)-[:HAS_STORE]->(s:STORE)-[u:USE]->(m:MONTH)\nWHERE c.name = '제주시' AND r.name = '삼도일동' AND s.MCT_TYPE = '베이커리'\nRETURN s.MCT_NM, u.WED_UE_CNT_RAT\nORDER BY u.WED_UE_CNT_RAT DESC\nLIMIT 1\n""")

MATCH (c:City)-[:HAS_REGION]->(r:Region)-[:HAS_STORE]->(s:STORE)-[u:USE]->(m:MONTH)
WHERE c.name = '제주시' AND r.name = '삼도일동' AND s.MCT_TYPE = '베이커리'
RETURN s.MCT_NM, u.WED_UE_CNT_RAT
ORDER BY u.WED_UE_CNT_RAT DESC
LIMIT 1



In [ ]:
print("""MATCH (c:City)-[:HAS_REGION]->(r:Region)-[:HAS_STORE]->(s:STORE)-[u:USE]->(m:MONTH)\nWHERE c.name = '제주시' AND r.name = '삼도일동' AND s.MCT_TYPE = '베이커리'\nRETURN s.MCT_NM, u.WED_UE_CNT_RAT\nORDER BY u.WED_UE_CNT_RAT DESC\nLIMIT 1""")

MATCH (c:City)-[:HAS_REGION]->(r:Region)-[:HAS_STORE]->(s:STORE)-[u:USE]->(m:MONTH)
WHERE c.name = '제주시' AND r.name = '삼도일동' AND s.MCT_TYPE = '베이커리'
RETURN s.MCT_NM, u.WED_UE_CNT_RAT
ORDER BY u.WED_UE_CNT_RAT DESC
LIMIT 1
